In [2]:

import pinocchio as pin
import numpy as np
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.visualize import (GepettoVisualizer, MeshcatVisualizer)
from sys import argv
import os
from os.path import dirname, join, abspath
 
# If you want to visualize the robot in this example,
# you can choose which visualizer to employ
# by specifying an option from the command line:
# GepettoVisualizer: -g
# MeshcatVisualizer: -m
VISUALIZER = MeshcatVisualizer
if False:
    opt = argv[1]
    if opt == '-g':
        VISUALIZER = GepettoVisualizer
    elif opt == '-m':
        VISUALIZER = MeshcatVisualizer
    else:
        raise ValueError("Unrecognized option: " + opt)
 
# Load the URDF model with RobotWrapper
# Conversion with str seems to be necessary when executing this file with ipython


mesh_dir = 'GraphIK/graphik/robots/urdfs'

urdf_model_path = "GraphIK/graphik/robots/urdfs/ur10_mod.urdf" #"GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
 
robot = RobotWrapper.BuildFromURDF(urdf_model_path, mesh_dir, pin.JointModelFreeFlyer())
 
# alias
model = robot.model
data = robot.data
 
# do whatever, e.g. compute the center of mass position expressed in the world frame
q0 = robot.q0
print(q0.shape)
q0[7:]= np.array([-83.86637669319315, -137.8137867825831, 168.9331609157763, 49.17494049988695, 42.556012244635504,0])*np.pi/180
#np.array([ -20.82406099 , -91.75986069 ,  45.3636569  , 103.75171042, -120.42082278,75.69289537, -101.3651413 ])*np.pi/180#np.array([0,0,0,0,20,0])*np.pi/180

com = robot.com(q0)
 
# This last command is similar to:
com2 = pin.centerOfMass(model,data,q0)
 
# Show model with a visualizer of your choice
if VISUALIZER:
    robot.setVisualizer(VISUALIZER())
    robot.initViewer()
    robot.loadViewerModel("pinocchio")
    robot.display(q0)

(13,)
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7043/static/


In [7]:
help(MeshcatVisualizer)

Help on class MeshcatVisualizer in module pinocchio.visualize.meshcat_visualizer:

class MeshcatVisualizer(pinocchio.visualize.base_visualizer.BaseVisualizer)
 |  MeshcatVisualizer(model=Nb joints = 1 (nq=0,nv=0)
 |    Joint 0 universe: parent=0
 |  , collision_model=None, visual_model=None, copy_models=False, data=None, collision_data=None, visual_data=None)
 |
 |  A Pinocchio display using Meshcat
 |
 |  Method resolution order:
 |      MeshcatVisualizer
 |      pinocchio.visualize.base_visualizer.BaseVisualizer
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  addGeometryObject(self, obj, color=None)
 |      Add a visual GeometryObject to the viewer, with an optional color.
 |
 |  captureImage(self, w=None, h=None)
 |      Capture an image from the Meshcat viewer and return an RGB array.
 |
 |  clean(self)
 |      Delete all the objects from the whole scene
 |
 |  delete(self, geometry_object, geometry_type)
 |
 |  disableCameraControl(self)
 |
 |  display(self, q=None)
 |

In [7]:
import pinocchio as pin
import numpy as np

from example_robot_data import load

# get panda robot usinf example_robot_data
mesh_dir = 'GraphIK/graphik/robots/urdfs'

urdf_model_path = "GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
#"GraphIK/graphik/robots/urdfs/ur10_mod.urdf"
#"GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"
 
robot = RobotWrapper.BuildFromURDF(urdf_model_path, mesh_dir, pin.JointModelFreeFlyer())

# get joint position ranges
q_max = robot.model.upperPositionLimit.T
q_min = robot.model.lowerPositionLimit.T
# get max velocity
dq_max = robot.model.velocityLimit
dq_min = -dq_max

# Use robot configuration.
q = (q_min+q_max)/2

## visualise the robot
from pinocchio.visualize import MeshcatVisualizer

viz = MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
# Start a new MeshCat server and client.
viz.initViewer(open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.display(q)
import time
viz.captureImage()

import pycapacity as pycap
import meshcat.geometry as g 


while True:
    # some sinusoidal motion
    for i in np.sin(np.linspace(-np.pi,np.pi,1000)):
        time.sleep(0.01)
        # update the joint position
        q[7] = i**2
        q[8] = i**1
        q[10] = i**2


        # calculate the jacobian
        data = robot.model.createData()
        pin.framesForwardKinematics(robot.model,data,q)
        pin.computeJointJacobians(robot.model,data, q)
        J = pin.getFrameJacobian(robot.model, data, robot.model.getFrameId(robot.model.frames[-1].name), pin.LOCAL_WORLD_ALIGNED)

        # visualise the robot
        viz.display(q)


f = '''
# use only position jacobian
J = J[:3,:]

# end-effector pose
Xee = data.oMf[robot.model.getFrameId(robot.model.frames[-1].name)]

# calculate the polytope
opt = {'calculate_faces':True}
# calculate the polytope
vel_poly = pycap.robot.velocity_polytope(J, dq_min, dq_max,options=opt)


# visualise the polytope and the ellipsoid
# meshcat triangulated mesh
poly = g.TriangularMeshGeometry(vertices=vel_poly.vertices.T/10 + Xee.translation, faces=vel_poly.face_indices)
viz.viewer['poly'].set_object(poly, g.MeshBasicMaterial(color=0x0022ff, wireframe=True, linewidth=3, opacity=0.2))

# calculate the ellipsoid
vel_ellipsoid = pycap.robot.velocity_ellipsoid(J, dq_max)
# meshcat ellipsoid
ellipsoid = g.Ellipsoid(radii=vel_ellipsoid.radii/10)
viz.viewer['ellipse'].set_object(ellipsoid, g.MeshBasicMaterial(color=0xff5500, transparent=True, opacity=0.2))
viz.viewer['ellipse'].set_transform(pin.SE3(vel_ellipsoid.rotation, Xee.translation).homogeneous)

'''


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7043/static/


KeyboardInterrupt: 

In [2]:
pip install serial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pybullet as p
import time
import math
from datetime import datetime
import pybullet_data

clid = p.connect(p.SHARED_MEMORY)
if (clid < 0):
  p.connect(p.GUI)
  #p.connect(p.SHARED_MEMORY_GUI)

p.setAdditionalSearchPath(pybullet_data.getDataPath())

p.loadURDF("plane.urdf", [0, 0, -0.3])
kukaId = p.loadURDF("kuka_iiwa/model.urdf", [0, 0, 0])
p.resetBasePositionAndOrientation(kukaId, [0, 0, 0], [0, 0, 0, 1])
kukaEndEffectorIndex = 6
numJoints = p.getNumJoints(kukaId)
if (numJoints != 7):
  exit()

#lower limits for null space
ll = [-.967, -2, -2.96, 0.19, -2.96, -2.09, -3.05]
#upper limits for null space
ul = [.967, 2, 2.96, 2.29, 2.96, 2.09, 3.05]
#joint ranges for null space
jr = [5.8, 4, 5.8, 4, 5.8, 4, 6]
#restposes for null space
rp = [0, 0, 0, 0.5 * math.pi, 0, -math.pi * 0.5 * 0.66, 0]
#joint damping coefficents
jd = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

for i in range(numJoints):
  p.resetJointState(kukaId, i, rp[i])

p.setGravity(0, 0, 0)
t = 0.
prevPose = [0, 0, 0]
prevPose1 = [0, 0, 0]
hasPrevPose = 0
useNullSpace = 1

useOrientation = 1
#If we set useSimulation=0, it sets the arm pose to be the IK result directly without using dynamic control.
#This can be used to test the IK result accuracy.
useSimulation = 1
useRealTimeSimulation = 0
ikSolver = 0
p.setRealTimeSimulation(useRealTimeSimulation)
#trailDuration is duration (in seconds) after debug lines will be removed automatically
#use 0 for no-removal
trailDuration = 15

i=0
while 1:
  i+=1
  #p.getCameraImage(320,
  #                 200,
  #                 flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX,
  #                 renderer=p.ER_BULLET_HARDWARE_OPENGL)
  if (useRealTimeSimulation):
    dt = datetime.now()
    t = (dt.second / 60.) * 2. * math.pi
  else:
    t = t + 0.01

  if (useSimulation and useRealTimeSimulation == 0):
    p.stepSimulation()

  for i in range(1):
    pos = [-0.4, 0.2 * math.cos(t), 0. + 0.2 * math.sin(t)]
    #end effector points down, not up (in case useOrientation==1)
    orn = p.getQuaternionFromEuler([0, -math.pi, 0])

    if (useNullSpace == 1):
      if (useOrientation == 1):
        jointPoses = p.calculateInverseKinematics(kukaId, kukaEndEffectorIndex, pos, orn, ll, ul,
                                                  jr, rp)
      else:
        jointPoses = p.calculateInverseKinematics(kukaId,
                                                  kukaEndEffectorIndex,
                                                  pos,
                                                  lowerLimits=ll,
                                                  upperLimits=ul,
                                                  jointRanges=jr,
                                                  restPoses=rp)
    else:
      if (useOrientation == 1):
        jointPoses = p.calculateInverseKinematics(kukaId,
                                                  kukaEndEffectorIndex,
                                                  pos,
                                                  orn,
                                                  jointDamping=jd,
                                                  solver=ikSolver,
                                                  maxNumIterations=100,
                                                  residualThreshold=.01)
      else:
        jointPoses = p.calculateInverseKinematics(kukaId,
                                                  kukaEndEffectorIndex,
                                                  pos,
                                                  solver=ikSolver)

    if (useSimulation):
      for i in range(numJoints):
        p.setJointMotorControl2(bodyIndex=kukaId,
                                jointIndex=i,
                                controlMode=p.POSITION_CONTROL,
                                targetPosition=jointPoses[i],
                                targetVelocity=0,
                                force=500,
                                positionGain=0.03,
                                velocityGain=1)
    else:
      #reset the joint state (ignoring all dynamics, not recommended to use during simulation)
      for i in range(numJoints):
        p.resetJointState(kukaId, i, jointPoses[i])

  ls = p.getLinkState(kukaId, kukaEndEffectorIndex)
  if (hasPrevPose):
    p.addUserDebugLine(prevPose, pos, [0, 0, 0.3], 1, trailDuration)
    p.addUserDebugLine(prevPose1, ls[4], [1, 0, 0], 1, trailDuration)
  prevPose = pos
  prevPose1 = ls[4]
  hasPrevPose = 1
p.disconnect()

Version = 4.1 Metal - 88
Vendor = Apple
Renderer = Apple M3 Pro


pybullet build time: Mar 20 2024 15:31:08


b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed


error: Not connected to physics server.

: 